### 0. 참고논문
* 머신러닝 기반 음식점 추천시스템 설계 및 구현 - 최환석, 팽전, 이 우 섭

### 1. Khaiii 설치
* !git clone https://github.com/kakao/khaiii.git
* !mkdir khaiii/build
* !cd khaiii/build; cmake ..
* !cd khaiii/build; make all
* !cd khaiii/build; make resource
* !cd khaiii/build; make package_python
* !cd khaiii/build/package_python; pip install .

In [1]:
!git clone https://github.com/kakao/khaiii.git
!mkdir khaiii/build
!cd khaiii/build; cmake ..
!cd khaiii/build; make all
!cd khaiii/build; make resource
!cd khaiii/build; make package_python
!cd khaiii/build/package_python; pip install .

Cloning into 'khaiii'...
remote: Enumerating objects: 1016, done.
remote: Counting objects: 100% (139/139), done.
remote: Compressing objects: 100% (104/104), done.
remote: Total 1016 (delta 46), reused 76 (delta 29), pack-reused 877
Receiving objects: 100% (1016/1016), 33.06 MiB | 22.92 MiB/s, done.
Resolving deltas: 100% (404/404), done.
-- [hunter] Initializing Hunter workspace (70287b1ffa810ee4e952052a9adff9b4856d0d54)
-- [hunter]   https://github.com/ruslo/hunter/archive/v0.23.34.tar.gz
-- [hunter]   -> /root/.hunter/_Base/Download/Hunter/0.23.34/70287b1
-- The C compiler identification is GNU 7.5.0
-- The CXX compiler identification is GNU 7.5.0
-- Check for working C compiler: /usr/bin/cc
-- Check for working C compiler: /usr/bin/cc -- works
-- Detecting C compiler ABI info
-- Detecting C compiler ABI info - done
-- Detecting C compile features
-- Detecting C compile features - done
-- Check for working CXX compiler: /usr/bin/c++
-- Check for working CXX compiler: /usr/bin/c++ -

### 2. 나눔폰트 설치
* !apt install fonts-nanum

#### [참고]
* 설치폴더: /usr/share/fonts/truetype/nanum
* download: !wget http://cdn.naver.com/naver/NanumFont/fontfiles/NanumFont_TTF_ALL.zip

In [2]:
!apt install fonts-nanum

Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following package was automatically installed and is no longer required:
  libnvidia-common-460
Use 'apt autoremove' to remove it.
The following NEW packages will be installed:
  fonts-nanum
0 upgraded, 1 newly installed, 0 to remove and 40 not upgraded.
Need to get 9,604 kB of archives.
After this operation, 29.5 MB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu bionic/universe amd64 fonts-nanum all 20170925-1 [9,604 kB]
Fetched 9,604 kB in 1s (8,035 kB/s)
Selecting previously unselected package fonts-nanum.
(Reading database ... 148489 files and directories currently installed.)
Preparing to unpack .../fonts-nanum_20170925-1_all.deb ...
Unpacking fonts-nanum (20170925-1) ...
Setting up fonts-nanum (20170925-1) ...
Processing triggers for fontconfig (2.12.6-0ubuntu2) ...


In [3]:
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.font_manager as fm
fm._rebuild()
# colab에서는 런타임 다시시작 필요
# ubuntu에서는 reboot

### 3. data 보기

In [1]:
import numpy as np
import pandas as pd

In [18]:
xl = pd.ExcelFile('데이터매쓰 영화 데이터.xlsx')

In [19]:
xl.sheet_names

['KOBIS_역대_박스오피스(통합전산망_집계_기준)_202']

In [20]:
df = pd.read_excel('데이터매쓰 영화 데이터.xlsx',skiprows=4)
import datetime
df.loc[1,'개봉일'] = df.loc[1,'개봉일'] - datetime.timedelta(days=365)
df.loc[55,'개봉일'] = df.loc[55,'개봉일'] - datetime.timedelta(days=365)
df.loc[20,'개봉일'] = df.loc[20,'개봉일'] - datetime.timedelta(days=365)
df.loc[41,'개봉일'] = df.loc[41,'개봉일'] - datetime.timedelta(days=365)
df.loc[47,'개봉일'] = df.loc[47,'개봉일'] - datetime.timedelta(days=365)
df.loc[76,'개봉일'] = df.loc[76,'개봉일'] - datetime.timedelta(days=365)


In [21]:
df.head(2)

,순위,영화명,개봉일,매출액,관객수,스크린수,상영횟수,대표국적,국적,배급사,장르,관람객,평론가,네티즌,남자,여자,10대,20대,30대,40대,50대 이상,OST,영상미,스토리,연기,연출,남자.1,여자.1,10대.1,20대.1,30대.1,40대.1,50대 이상.1,OST.1,영상미.1,스토리.1,연기.1,연출.1,남자,여자.2,...,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50
0,1,명량,2014-07-30,135757418810,17615686,1587,188720,한국,한국,(주)씨제이이엔엠,"액션, 드라마",8.88,6.29,8.44,8.81,8.94,9.19,8.92,8.76,8.83,8.82,0,0,50,50,0,7.47,9.08,8.72,8.21,7.65,8.03,7.88,11,18,22,26,24,53,47,...,"오늘 소개할 영화는 ""명량""입니다\n\n2014년에 개봉하여 천만 관객을 가뿐히 넘...","​명량은 1597년, \n\n백의종군 중이던 이순신 장군이 다시 삼도수군통제사로 임...","영화 <명량>의 배경은 역사적인 사실에 바탕을 하고 있어요. 1597년 정유재란,...",오랜만에 영화 <명량>을 또 보았다. 2014년 개봉한 <명량>은 그 유명세와 달리...,이순신 장군의 모습을 보여준 영화 <명량>입니다.\n\n천칠백만명의 관객이 봤었는데...,"영화보는걸 나름 좋아한다고 생각했는데\n\n취향이 확고한건지, 배우를 많이 보는건지...",개봉 당시에도 정말 재미있게 봤었고\n\n후에 몇번 더 볼 때도\n\n감탄하면서 봤...,2.'영화'가 가진 매력\n \n①영웅 이순신의 면모\n\n영화 <명량>에서는 ...,"[亂中日記 二]\n\n 죽고자 하면 반드시 살 것이요,\n\n 살고자 하면 반드시...",영화는 영화였다\n\n역사적 사실과 다른 부분이 있기는 하나\n그 패턴은 고증을 토...,"""저는 명량을 보고 나서 처음 든 느낌은 ""이게 바로 영화의 매력이지!""라는 생각이...","영화 <명량>은 1597년, 단 12척의 배로 330척 왜군의 공격에 맞서 싸운 이...","명량, 백성이 있어야 나라가 있고, 나라가 있어야 임금이 있는 법이지 살고자 하면 ...","개봉한 지 약 7년만에 처음으로 명량을 보았다.\n\n늘 보고 싶었는데, 이번에 한...",광남이가 영화 명량을 통해 또 한번 느낀 점은 현재가 있을 수 있는 이유는 과거가 ...,너무나도 유명한 영화였지만 나는 이제야 보게 되었다. “노희영의 브랜딩 법칙”이라...,여러분 안녕하세요 행복한 수요일입니다~! 다들 즐거운 하루 보내고들 계신가요~?ㅎㅎ...,영화 명량은 조선시대 실존 인물인 이순신장군이 임진왜란때\n\n12척의 배로 330...,이 명량이라는 영화를 보고 내 머리에 남은 강렬한 기억은 구루지마의 머리를 이순신이...,조선을 대표해서 전쟁에 나가는 이순신장군과 일본을 대표해서 전쟁에 나가는\n구루지마...,이번에 소개드릴 영화는 2014년에 개봉한 <명량>입니다.\n영화관에 개봉되면서 작...,오늘은 한국 역사 영화 추천 명량 리뷰를 해볼까 합니다.\n\n우리나라 사람이라면 ...,"""신에게는 아직 열두척의 배가 있사옵니다.""\n\n누구나 잘 아는 아주 유명한 대사...",김한민 감독의 2014년 작품이다.\n\n\n\n 이순신 장군의 명량해전을 그린 ...,"영화 명량은 우리나라의 대표 위인 이순신 장군의 명량해전을 소재로\n한 작품인데요,...",때마침 넷플릭스에서 영화목록에 명량이 있길래\n\n오랜만에 다시 보게되었는데\n\n...,영화 ‘명량’은1597년 임진왜란시 12척의 배로 300여척에 달하는 왜군의 공격...,일단 명량을 정확하게 이해하시려면 명량대첩에 대해서 간단히 알고 있다면 도움이 되겠...,​<명량>은 우리의 역사 영화인데요\n\n​\n\n현재까지도 광화문 광장에\n\n굳...,영화 명량은 이순신 장군의 역의 맡은 \n\n 배우 최민식의 연기가 돋보였다.\n\...,명량을 봤습니다. 유튜브 돌리다가 우연히 불멸의 이순신 클립 영상을 봤는데 cg가 ...,며칠동안 많은 고민 끝을 했다. 그 동안 꽤 많은 영화를 보았는데 과연 '어떤 영화...,모두들 알고 있듯이 1597년 임진왜란이 일어난지 6년\n\n매우 오래 지속된 전쟁...,"2014년 대종상 시상식에서 최우수작품상, 남우 주연상, 기획상, 기술상을,\n\n...","영화 명량, 참 오랜만에 다시보는 영화입니다. 6년전 이 영화가 개봉될 때 1700...","‘필사즉생 필생즉사’. 죽고자 하면 살 것이고, 살고자 하면 죽을 것이라는 뜻의 이...",오늘은 예전에 보았던 영화 중에 정말 재미있게 보았고 의미 있는 영화라고 \n\n생...,"국민의 존경을 한 몸에 받는 이순신,\n그는 누구를 위해서 싸웠는지 아직도 감이 잡...",명량해전의 승리는 영화에서 나오는 것처럼\n몇몇 백성과 장수들의 노력이나 수군덕만은...,당시 1000만명이나 넘긴 상당해서 리뷰 지금부터 관객수도 명량 바로 영화는 명량인...
1,2,극한직업,2018-01-23,139655543516,16266338,2003,292816,한국,한국,(주)씨제이이엔엠,코미디,9.2,6.80,8.5,9.16,9.24,9.42,9.32,9.22,9.08,8.82,0,0,31,38,31,7.63,8.71,8.79,8.33,8.08,7.88,7.59,8,8,25,35,24,45,55,...,(3) 코미디\n\n​\n\n​\n\n\n보통 한국 영화 중 코미디 영화들을 보면 ...,저는 외국영화보다는 한국영화를 선호하는 편이에요\n장르는 로맨스나 코미디를 굉장히 ...,이번 설날 연휴 특선 영화 편성표를 봤을 때 <사바하>랑 함께 이 영화는 시간 놓치...,안녕하세요 독고 입니다\n\n\n\n오늘은 바로 오늘 개봉한 따끈따끈한 신작 영화 ...,지금까지 이런 맛은 없었다...\n\n갈비인가? 치킨인가?ㅋㅋㅋㅋ\n\n이미 이 대...,"작정하고 돌아온 이병헌 감독, 그리고 작정하고 영화 찍은 다섯배우 류승룡, 이하늬,...",영화 <극한직업>의 가장 큰 매력포인트는 극의 중심이 되는 마약반 5인방의 환상적인...,내 평점 ★☆\n\n​\n\n오랜만에 올리누\n\n​\n\n본인은 코미디 영화 성애...,걸걸한 여형사 '장형사'의 이하늬 님과 중국교포 출신의 '마형사'의 진선규 님의 케...,"결론부터 말하자면,\n\n영화를 보낸 내내 계속 웃으면서 봤습니다!! 간만에 영화를...","이병헌 감독의 영화 '극한직업'은 2019년 1월 23일에 개봉해 1,626만 명...",오늘은 치킨을 뜯으면서 보기 좋은 영화\n\n극한직업을 소개해보려 해요\n\n​\n...,낮에는 치킨장사! 밤에는 잠복근무!\n\n지금까지 이런 수사는 없었다!\n\n불철주...,저는 극장에서도 보고 후에 넷플릭스로 한번 더 보기도 한 작품이었습니다. \n\n킬...,<극한직업>의 줄거리는 '범인을 잡기 위해 얼떨결에 시작한 치킨집이 대박난다'로 요...,우선 나는 너무 재밌게 보고왔다. 사실 코미디영화가 관객들을 끊임없이 웃게 만들었다...,<극한직업>이 개봉 당시 정말 붐이었어서\n\n어쩌면 명량을 제칠 수도 있겠다고 생...,영화관에서 직접 봤던 극한 직업. 만족스러운 기분을 느끼고 나왔다. \n\n소재도 ...,극한직업이라는 영화를 보고왔습니다..\n\n장르는 코미디구요..\n\n영화보는내내 ...,영화 극한직업 리뷰 엔딩크레딧 쿠키영상 없음 포스팅 시작합니다.\n\n우선 쿠키영상...,영화 극한직업의 줄거리를\n\n내 개인적인 생각을 정리해보자.\n\n​\n\n고반장...,본지는 오래됐지만 뒤늦게 리뷰를 남겨봅니다.\n\n저는 입소문이 무진장 많이 탔을 ...,이병헌 감독의 영화\n\n2015년 영화 <스물>로 자신의 영화를 많은데에 알린 이...,‘극한직업’ 후기를 작성하려고 \n\n맥북을 폈는데\n\n무슨 말을 써야 하는지 참...,한국형 코미디 형식 그대로 풀어 \n\n가족 3대가 모두 가볍게 볼 수 있는\n\n...,개봉한지 한참 지난 영화를 

In [22]:
df.shape

(100, 95)

### 4. review 받기

In [23]:
import urllib
from urllib.request import urlopen
from bs4 import BeautifulSoup
import re

In [24]:
def get_review_content_url(title,year):
    baseurl = 'https://movie.naver.com/movie/search/result.naver?query=%s&section=movie&ie=utf8'
    pagenum = 1
    firsturl = baseurl % (urllib.parse.quote(title),)
    reviewurl = None
    while True:
        if pagenum == 1:
            url = firsturl
        else:
            url = '%s&page=%d' % (firsturl,pagenum)
#         print(url)
        data = urlopen(url).read()
        html = BeautifulSoup(data, 'html5lib')
        ul = html.find_all(attrs={'class': 'search_list_1'})
        if len(ul)>0:
            lilist = ul[0].find_all('li')
            for li in lilist:
                dt = li.find_all('dt')
                a1 = dt[0].find('a')
                t = a1.text
                dd = li.find_all(attrs={'class': 'etc'})
                a = dd[0].find_all('a')
                y = 0
                if re.search(r'^\d+$',a[-1].text):
                    y = int(a[-1].text)
                mlen = min(len(t),len(title))
                if title[:mlen] == t[:mlen]:
                    if y == year:
                        reviewurl = 'https://movie.naver.com' + re.sub('basic','review',a1.attrs['href'])
                        break
            if reviewurl is not None:
                break
        else:
            break
        pagenav = html.find(attrs={'class': 'pagenavigation'})
        tdlist = pagenav.find_all('td')
        lasttd = tdlist[-1]
        strnext = lasttd.text
        if strnext != '다음':
            break
        pagenum = pagenum+1
    if reviewurl is None:
        return reviewurl,None
    else:
        return reviewurl,t

def get_code(reviewurl):
    code = re.search(r'(?<=code=)[0-9]+',reviewurl).group(0)
    return code

def get_nid_list(reviewurl,maxreviews=None):
    nidlist = []
    page = 1
    while True:
        if page == 1:
            url = reviewurl
        else:
            url = '%s&page=%d' % (reviewurl,page)
        data = urlopen(url).read()
        html = BeautifulSoup(data, 'html5lib')
        ul = html.find(attrs={'class': 'rvw_list_area'})
        if ul is None:
            return []
        lilist = ul.find_all('li')
        for li in lilist:
            a = li.find_all('a')
            num = a[0].attrs['onclick']
            if not a[0].attrs.get('onclick'):
              continue
            nid = re.search(r'\d+',num).group(0)
            nidlist.append(nid)
            if maxreviews != None and len(nidlist) >= maxreviews:
                return nidlist
        paging = html.find(attrs={'class': 'paging'})
        a = paging.find_all('a')
        if a[-1].text != '다음':
            break
        page = page + 1
    
    return nidlist

def get_review_list(nidlist,code):
    reviewprefix = 'https://movie.naver.com/movie/bi/mi/reviewread.naver'
    review_list = ['%s?nid=%s&code=%s&order=#tab'%(reviewprefix,x,code) for x in nidlist]
    return review_list

def get_review_text(review_list):
    review_texts = []
    scores = []
    titles = []
    nicknames = []
    clicks = []
    recommands = []
    dates = []
    for url in review_list:
        data = urlopen(url).read()
        html = BeautifulSoup(data, 'html5lib')
        tx_area = html.find(attrs={'class': 'user_tx_area'})
        star_score = html.find(attrs={'class': 'star_score'})
        if star_score is not None:
          star_score = star_score.find('em')
        h_lst_tx = html.find(attrs={'class': 'h_lst_tx'})
        review_texts.append(tx_area.text.strip())
        name = html.find(attrs={'class': 'board_title'}).find_all('li')[-1].find('a').find('em')
        user_tx_info = html.find(attrs={'class': 'user_tx_info'}).find_all('span')
        wrt_date = html.find(attrs={'class': 'wrt_date'})
        if star_score is None:
          scores.append(None)
        else:
          scores.append(star_score.text)
        titles.append(h_lst_tx.text)
        nicknames.append(name.text)
        clicks.append(user_tx_info[0].find('em').text)
        recommands.append(user_tx_info[1].find('em').text)
        dates.append(wrt_date.text)

    print(len(nicknames),)
    df = pd.DataFrame({'별명': nicknames, '리뷰평점': scores, '리뷰조회': clicks,'리뷰추천': recommands,
                       '리뷰날짜': dates, '리뷰제목': titles, '리뷰': review_texts})
    return df

In [25]:
rnum = []
for i,row in df.iterrows():
    # 영화 선택
    title = row['영화명']
    year = row['개봉일'].year
    n = 0
    for j in range(1,51):
        a = row[j]
        if type(a) != str:
            n += 1
    rnum.append(n)

In [26]:
rnum

[0,
 0,
 0,
 20,
 20,
 50,
 50,
 50,
 50,
 50,
 20,
 20,
 48,
 20,
 50,
 20,
 20,
 20,
 20,
 23,
 20,
 50,
 50,
 50,
 20,
 50,
 50,
 50,
 50,
 50,
 50,
 50,
 50,
 50,
 50,
 50,
 50,
 50,
 50,
 50,
 20,
 20,
 20,
 20,
 20,
 20,
 20,
 20,
 20,
 20,
 50,
 50,
 50,
 50,
 50,
 50,
 50,
 50,
 50,
 50,
 50,
 50,
 50,
 50,
 50,
 50,
 50,
 50,
 50,
 50,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 20,
 20,
 20,
 50,
 50,
 50,
 50,
 50,
 50,
 50,
 50,
 50,
 50,
 50,
 50,
 50,
 50,
 50,
 50,
 50,
 50,
 50,
 50]

In [27]:
check_next_year = True
reviewurllist = []
for i,row in df.iterrows():
    # 영화 선택
    title = str(row['영화명'])
    year = row['개봉일'].year
    years = [year]
    if check_next_year:
        years = [year,year+1,year-1]
    for y in years:
        reviewurl,title2 = get_review_content_url(title,y)
        if reviewurl is not None:
            break
    reviewurllist.append([str(i)+'. '+title+' - '+title2,str(year),reviewurl])

In [28]:
[x[0] for x in reviewurllist]

['명량-명량 (ROARING CURRENTS)',
 '극한직업-극한직업 (Extreme Job)',
 '신과함께-죄와 벌-신과함께-죄와 벌 (Along With the Gods: The Two Worlds)',
 '국제시장-국제시장 (Ode to My Father)',
 '어벤져스: 엔드게임-어벤져스: 엔드게임 (Avengers: Endgame)',
 '겨울왕국 2-겨울왕국 2 (Frozen 2)',
 '베테랑-베테랑 (Veteran)',
 '도둑들-도둑들 (The Thieves)',
 '7번방의 선물-7번방의 선물 (Miracle in Cell No.7)',
 '알라딘-알라딘 (Aladdin)',
 '암살-암살 (Assassination)',
 '광해, 왕이 된 남자-광해, 왕이 된 남자 (Masquerade)',
 '신과함께-인과 연-신과함께-인과 연 (Along with the Gods: The Last 49 Days)',
 '택시운전사-택시운전사 (A Taxi Driver)',
 '부산행-부산행 (TRAIN TO BUSAN)',
 '변호인-변호인 (The Attorney)',
 '어벤져스: 인피니티 워-어벤져스: 인피니티 워 (Avengers: Infinity War)',
 '어벤져스: 에이지 오브 울트론-어벤져스: 에이지 오브 울트론 (The Avengers: Age of Ultron)',
 '인터스텔라-인터스텔라 (Interstellar)',
 '기생충-기생충 (PARASITE)',
 '겨울왕국-겨울왕국 (Frozen)',
 '보헤미안 랩소디-보헤미안 랩소디 (Bohemian Rhapsody)',
 '검사외전-검사외전 (A Violent Prosecutor)',
 '엑시트-엑시트 (EXIT)',
 '설국열차-설국열차 (Snowpiercer)',
 '관상-관상 (The Face Reader)',
 '아이언맨 3-아이언맨 3 (Iron Man 3)',
 '캡틴 아메리카: 시빌 워-캡틴 아메리카: 시빌 워 (Captain America: Civil Wa

In [13]:
df.loc[df.영화명=='극한직업']

,순위,영화명,개봉일,매출액,관객수,스크린수,상영횟수,대표국적,국적,배급사,장르,관람객,평론가,네티즌,남자,여자,10대,20대,30대,40대,50대 이상,OST,영상미,스토리,연기,연출,남자.1,여자.1,10대.1,20대.1,30대.1,40대.1,50대 이상.1,OST.1,영상미.1,스토리.1,연기.1,연출.1,남자,여자.2,...,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50
1,2,극한직업,2019-01-23,139655543516,16266338,2003,292816,한국,한국,(주)씨제이이엔엠,코미디,9.2,6.8,8.5,9.16,9.24,9.42,9.32,9.22,9.08,8.82,0,0,31,38,31,7.63,8.71,8.79,8.33,8.08,7.88,7.59,8,8,25,35,24,45,55,...,(3) 코미디\n\n​\n\n​\n\n\n보통 한국 영화 중 코미디 영화들을 보면 ...,저는 외국영화보다는 한국영화를 선호하는 편이에요\n장르는 로맨스나 코미디를 굉장히 ...,이번 설날 연휴 특선 영화 편성표를 봤을 때 <사바하>랑 함께 이 영화는 시간 놓치...,안녕하세요 독고 입니다\n\n\n\n오늘은 바로 오늘 개봉한 따끈따끈한 신작 영화 ...,지금까지 이런 맛은 없었다...\n\n갈비인가? 치킨인가?ㅋㅋㅋㅋ\n\n이미 이 대...,"작정하고 돌아온 이병헌 감독, 그리고 작정하고 영화 찍은 다섯배우 류승룡, 이하늬,...",영화 <극한직업>의 가장 큰 매력포인트는 극의 중심이 되는 마약반 5인방의 환상적인...,내 평점 ★☆\n\n​\n\n오랜만에 올리누\n\n​\n\n본인은 코미디 영화 성애...,걸걸한 여형사 '장형사'의 이하늬 님과 중국교포 출신의 '마형사'의 진선규 님의 케...,"결론부터 말하자면,\n\n영화를 보낸 내내 계속 웃으면서 봤습니다!! 간만에 영화를...","이병헌 감독의 영화 '극한직업'은 2019년 1월 23일에 개봉해 1,626만 명...",오늘은 치킨을 뜯으면서 보기 좋은 영화\n\n극한직업을 소개해보려 해요\n\n​\n...,낮에는 치킨장사! 밤에는 잠복근무!\n\n지금까지 이런 수사는 없었다!\n\n불철주...,저는 극장에서도 보고 후에 넷플릭스로 한번 더 보기도 한 작품이었습니다. \n\n킬...,<극한직업>의 줄거리는 '범인을 잡기 위해 얼떨결에 시작한 치킨집이 대박난다'로 요...,우선 나는 너무 재밌게 보고왔다. 사실 코미디영화가 관객들을 끊임없이 웃게 만들었다...,<극한직업>이 개봉 당시 정말 붐이었어서\n\n어쩌면 명량을 제칠 수도 있겠다고 생...,영화관에서 직접 봤던 극한 직업. 만족스러운 기분을 느끼고 나왔다. \n\n소재도 ...,극한직업이라는 영화를 보고왔습니다..\n\n장르는 코미디구요..\n\n영화보는내내 ...,영화 극한직업 리뷰 엔딩크레딧 쿠키영상 없음 포스팅 시작합니다.\n\n우선 쿠키영상...,영화 극한직업의 줄거리를\n\n내 개인적인 생각을 정리해보자.\n\n​\n\n고반장...,본지는 오래됐지만 뒤늦게 리뷰를 남겨봅니다.\n\n저는 입소문이 무진장 많이 탔을 ...,이병헌 감독의 영화\n\n2015년 영화 <스물>로 자신의 영화를 많은데에 알린 이...,‘극한직업’ 후기를 작성하려고 \n\n맥북을 폈는데\n\n무슨 말을 써야 하는지 참...,한국형 코미디 형식 그대로 풀어 \n\n가족 3대가 모두 가볍게 볼 수 있는\n\n...,개봉한지 한참 지난 영화를 리뷰하는 시간 바로 뒷북 리뷰입니다 이 영화는 2019년...,"여태까지 많이봤다면 꽤 많은 \n\n범죄수사극, 범죄코메디, 범죄영화, 코메디영화들...",웃음 하나에만 주력한 영화\n\n재미있다. 그냥 우리는 감독이 의도한 개그 코드에 ...,제가 이 영화를 추천하는 이유는 오랜만에 재미있게 웃을 수 있었던 코미디 영화여서인...,주말에 보기 정말 좋은 영화였어요.\n\n​\n\n쉬는 날 잠옷입고 치킨먹으면서 딩...,"설연휴를 기념하여 오랜만에 극장을 찾았다. 맨날 보려고 했다가 취소하고, 보려고 했...",영화 극한직업은 매우 훌륭한 레시피로 만든 웰메이드 요리에 비유할 수 있겠다. 영화...,이 영화를 2번봤는데요. 스토리는 극한직업은 형사들이 치..(서론 생략)\n\n\n...,영화 극한직업 평점주기\n\n재미 ★★★★★\n\n연기력 ★★★★★\n\n내용구성 ...,우울한 겨울 끝에 여기저기서 들려오는 소리. \n\n \n\n“극한직업 재미있어요....,영화관에서 최근에 한국영화를 본 게 판도라였데 오랜만에 한국영화를 봤습니다. 마약범...,이번에 보고 온 영화는 2019년 1월 23일에 개봉한 한국 코미디 영화 <극한직업...,영화를 보는 시간은 크게 두가지가 있다고 생각한다. 영화라는 창작물 혹은 예술작품을...,"정말 아무 생각 없이 재미있다는 압도적인 평가 때문에, 설 연휴에 영화라도 한 편 ...",새해를 여는 웃음 바이러스! 마침 설연휴와 겹쳐 설명절 증후군을 겪는 사람들에게도 ...


In [29]:
reviewurllist[:2]

[['명량-명량 (ROARING CURRENTS)',
  '2014',
  'https://movie.naver.com/movie/bi/mi/review.naver?code=93756'],
 ['극한직업-극한직업 (Extreme Job)',
  '2018',
  'https://movie.naver.com/movie/bi/mi/review.naver?code=167651']]

In [30]:
[x for x in reviewurllist if x[2] is None]

[]

In [31]:
nidlist = []
for i,a in enumerate(reviewurllist):
    title,year,reviewurl = a
    code = get_code(reviewurl)
    nreviews = rnum[i]
    print(i, '--')
    if nreviews ==0:
        continue
    nidlist = get_nid_list(reviewurl,maxreviews=nreviews)
    review_list = get_review_list(nidlist,code)
    df_review = get_review_text(review_list)
    k = 0
    for j in range(1,31):
        b = df.loc[i,j]
        if type(b) != str:
            df.loc[i,j] = df_review.loc[k,'리뷰']
            k += 1
    print(k)

0 --
1 --
2 --
3 --
20
4 --
20
5 --
50
6 --
50
7 --
50
8 --
50
9 --
50
10 --
20
11 --
20
12 --
48
13 --
20
14 --
50
15 --
20
16 --
20
17 --
20
18 --
20
19 --
23
20 --
20
21 --
50
22 --
50
23 --
50
24 --
20
25 --
50
26 --
50
27 --
50
28 --
50
29 --
50
30 --
50
31 --
50
32 --
50
33 --
50
34 --
50
35 --
50
36 --
50
37 --
50
38 --
50
39 --
50
40 --
20
41 --
20
42 --
20
43 --
20
44 --
20
45 --
20
46 --
20
47 --
20
48 --
20
49 --
20
50 --
50
51 --
50
52 --
50
53 --
50
54 --
50
55 --
50
56 --
50
57 --
50
58 --
50
59 --
50
60 --
50
61 --
50
62 --
50
63 --
50
64 --
50
65 --
50
66 --
50
67 --
50
68 --
50
69 --
50
70 --
71 --
72 --
73 --
74 --
75 --
76 --
77 --
20
78 --
20
79 --
20
80 --
50
81 --
50
82 --
50
83 --
50
84 --


KeyError: ignored

In [32]:
nidlist

['4473519',
 '4478537',
 '4479266',
 '4478610',
 '4478138',
 '4481559',
 '4478200',
 '4478729',
 '4476747',
 '4478549',
 '4490814',
 '4475037',
 '4473097',
 '4479515',
 '4479571',
 '4488892',
 '4476270',
 '4480061',
 '4475200',
 '4477961',
 '4484177',
 '4524858',
 '4556524',
 '4489955',
 '4478756',
 '4479485',
 '4483387',
 '4491763',
 '4488461',
 '4478992',
 '4482744',
 '4479228',
 '4477990',
 '4480119',
 '4479265',
 '4478476',
 '4482724',
 '4485486',
 '4479609',
 '4491708',
 '4521017',
 '4488195',
 '4490118',
 '4496709',
 '4481431',
 '4481959',
 '4586908',
 '4480782',
 '4486591',
 '4481522']

In [33]:
df.loc[3]

순위                       4
영화명                   국제시장
개봉일    2014-12-17 00:00:00
매출액           110944607730
관객수               14263980
              ...         
46                     NaN
47                     NaN
48                     NaN
49                     NaN
50                     NaN
Name: 3, Length: 95, dtype: object

In [34]:
df.to_csv('데이터매쓰 영화 데이터 완성.csv', index=False)

In [ ]:

nreviews,k

In [ ]:
from google.colab import drive
drive.mount('gdrive')

Mounted at gdrive


In [ ]:
df_review.to_csv('gdrive/MyDrive/a.csv')

In [ ]:
reviewurl

'https://movie.naver.com/movie/bi/mi/review.naver?code=149269'

In [ ]:
i

55

In [ ]:
df.loc[i]

순위                      56
영화명                     사도
개봉일    2015-09-16 00:00:00
매출액            48846723501
관객수                6247745
              ...         
46                     NaN
47                     NaN
48                     NaN
49                     NaN
50                     NaN
Name: 55, Length: 95, dtype: object

In [ ]:
df.loc[55,'개봉일'] = df.loc[55,'개봉일'] - datetime.timedelta(days=365)

Timestamp('2014-09-16 00:00:00')

In [ ]:
import datetime
df.loc[55,'개봉일'] = df.loc[55,'개봉일'] - datetime.timedelta(days=365)

datetime.timedelta(days=365)